In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
# ⚠️ 修复 1：引入专为聊天模型设计的 FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# 1. 必备起手式：强制加载你本地的 API 密钥
load_dotenv(override=True)

# 2. 准备几个“打样”的示例 (Few-Shot Examples)
examples = [
    {"input": "India", "output": "aidnI"},
    {"input": "China", "output": "anihC"},
    {"input": "United States", "output": "setatS detinU"},
    {"input": "Shenzhen", "output": "nehznehS"}
]

# 3. 定义这些示例如何转化为对话格式
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"), 
    ("ai", "{output}")
])

# 4. ⚠️ 修复 2：使用最新的聊天专用模板类组装示例
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 5. 拼装终极提示词流水线：系统人设 + 打样示例 + 你的实际问题
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that reverses the input string."),
    few_shot_prompt,
    ("human", "{input}")
])

# 6. 动态填入变量 
prompt = prompt_template.format_messages(input="Brazil")
print("组装给大模型的底层消息流长这样：\n", prompt, "\n")

# 7. 唤醒大模型 (温度设为 0，因为翻转字母是严谨任务，不需要创造力)
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 8. 发送请求并规范打印结果
print("正在等待大模型处理...\n")
response = model.invoke(prompt)
print("最终的翻转结果:", response.content)

组装给大模型的底层消息流长这样：
 [SystemMessage(content='You are a helpful assistant that reverses the input string.', additional_kwargs={}, response_metadata={}), HumanMessage(content='India', additional_kwargs={}, response_metadata={}), AIMessage(content='aidnI', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='China', additional_kwargs={}, response_metadata={}), AIMessage(content='anihC', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='United States', additional_kwargs={}, response_metadata={}), AIMessage(content='setatS detinU', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='Shenzhen', additional_kwargs={}, response_metadata={})] 

正在等待大模型处理...

最终的翻转结果: nehzneS
